In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
res1 = pd.read_csv("../data/results.csv")
res2 = pd.read_csv("../data/results_pi.csv")

ParserError: Error tokenizing data. C error: Expected 11 fields in line 8, saw 18


In [3]:
res1

,,,,,,,,,,,,,,,timestamp,address,value
2020-08-06 14:25:03.329896,['3AVLMuAgA8Bm9mz5JPZx1sw2gcfHxkMqTE','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','32MrcgjUTzcpb4jbrU8TKRSdPFzXcxhrce','3CRMc76siFi4BPxrfs2ML77573gENaf9yp','3MUiVqL3fqT9JMmnvWe6HgZRqFSW2onfjb','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','3MWfFfZ8Ke9PevNPqVN9wUr4SinP6JNWiV','37Z9c9bKri9Qo8GGAhGRbd1FBZn7CGneBw','34oWp1asVonAjKen1NrqMi8vEUDvyPWN5e','1C2Ki4nTuUjJbK48zNd2Um2FT4PUTB56ss','3EjCEJGvujDDGEUSA7gWo5PHZTuXWHvRJG','3KMHqr3A8iDwDkuWrXVrUfB6RdQjYekGZW','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y'],3.000003e+09
2020-08-06 14:28:47.552292,['3DdVsZrbzaVJny56UAR5BgsXdWiZoyo7wZ','3Dd2mMng3RTmu1PyPZns8FVEe1iZPp6UNj','3Dg7yCA91ko3d6Voase6Wb537pr4Kun5Um','3Dg6xvMFbY9fu12yjKNTdHcv1u2dBU2Mck','3DcSJH3mSGfW9GGvkGTt8pmaxtW1f1UfsK','3Dfd36geNT35tx42AUFJNrDY7JjSJJbrHf','3DfsYHqorT1tGrVv8e2mQMGR121GdHJgXS','3Db1bAfxPkoxVeoG5peFfi8Z4yc42xfd6C','3DcpATgdDVxReQuRYFBNUnJKd6UYtHyxKf','3DfXPKXj7nsLYQvr9z7am1XCGJ42Ush9Kj','3DgPTxeLVeiLJkAik9c9H56FYsJTtYKUSn','3DfcXBESU5jCjPRk2gEoZVjqtHGqQYTkB2','3Dd9gsnGsv8vMPRXnHWWhUfeoUArGgw8KV','3DdDW85C961g79RdHoMcvXKNe8e7FKjJCr','3DfZ5vpF84Rhb8Kr5kXXv5aRTJfGy3v1Em','3DcCE5vDnzKsyQaFK8rUsT9CJEdHjoyEj9'],1.683815e+09
2020-08-06 14:29:37.736201,['3GLrQ3tTpxQoHdzb6yd74jrhU22sLwZZKh'],11971431236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-06 14:31:02.032051,['1XYZJsM6cnyuDMVnfFufb8dzVM7cS5rqd'],19999876016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-06 14:31:02.573763,['1Kr6QSydW9bFQG1mXiPNNu6WpJGmUa9i1g','32Bx8GY8zGzVn5hAxzqV1JBAWGd6vmAKUg'],23493932386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-06 14:41:24.155676,['3HvdRx2aqYzqYQwKaA12VtdRPirRfuDwFy','339JpaofJvEvrtYnLbFJwNaM5ysgKmCAb5','3HvdRx2aqYzqYQwKaA12VtdRPirRfuDwFy','339JpaofJvEvrtYnLbFJwNaM5ysgKmCAb5','3K39ZwHJXR5627A3FpSwjds81i7dHPK34E','3AvB9Hx2kCirz7KYZwZhFjMPZWSFQEK4UT','3CUyZVp3T47DHr4oBEG93aZXQ35UMmtc2n','3KUJGuyCrqg39de5a4H3GQybANtc9nZqBn','3N6Es6ERicanAQBMu2uNXy3nbZjgT8E74W','3HvdRx2aqYzqYQwKaA12VtdRPirRfuDwFy','3N6Es6ERicanAQBMu2uNXy3nbZjgT8E74W','3CUyZVp3T47DHr4oBEG93aZXQ35UMmtc2n','3CUyZVp3T47DHr4oBEG93aZXQ35UMmtc2n','3CUyZVp3T47DHr4oBEG93aZXQ35UMmtc2n','3MdpKiUEZahbTXhTBBndSJ2K3fFvLvjP7r','1HAVuWEM4b5jozhP6rY76T56kzveqD9avy'],5.000000e+09
2020-08-06 14:41:35.756134,['3KDpBGSXm75BmGpu1noZRQo1wzuZTYWS6y','39cr5LGZekw8BEeZBdKxKa4qXDDZVKZqNG','36UoAE2EqgfKseC9JEBNJKgTt4XDnv8wNx','3EnwZKc49LvKYTABtXuW5J1GwQs26ufbqo','3CUyZVp3T47DHr4oBEG93aZXQ35UMmtc2n','1JfXunNT8Hjf9jT8beo2XiRMomFejpX1UU','1KQoWZYd4dBamAUQGVks6pyZh4bsaWQqt2','1KQoWZYd4dBamAUQGVks6pyZh4bsaWQqt2','3MdpKiUEZahbTXhTBBndSJ2K3fFvLvjP7r','3MdpKiUEZahbTXhTBBndSJ2K3fFvLvjP7r','3CUyZVp3T47DHr4oBEG93aZXQ35UMmtc2n','3NURjncwGsMY4aWLwSxshNvKsaQRKp27Lx','36dSJ2ExAn1jbnV5HreYSMMfcYHXzouuFT','3Lh8K5xpGxp9JbPCyAVn79Sk7ZahKtCJS7','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','1HVn2MHLbh32g1PvoYVnz51xPqWZKutach'],5.000000e+09
2020-08-06 14:41:39.090250,['36UoAE2EqgfKseC9JEBNJKgTt4XDnv8wNx','3BZR7EzhDoDwAj1A46tH5K59d5Vxux6xnz','33HgF4NCKf85aNMHsdBfxr4xF28iQBX9HN','34P2FShiNxtMxoGJAHoyqPrTjhjb1t9bxa','3DCvWdLgDDVrpGXXgew5s94j7kwBB7D1Dm','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','1KQoWZYd4dBamAUQGVks6pyZh4bsaWQqt2','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','3Fa49WMAFzAo1d5yJdSgv95A7DN64apzos','12QY6M2zf1P9AJUXbxdzVLt2G5iqUCsB3y','3AnsQvAhXu4ewvSDTpHz4wLhdLQJTALUDC','35iNvGVDLnYnVPnt6xHqHznhMDJ5o68Z4c','3Ahz2oa94hCc57XnCTe3BfswUXQfubuyWh','38KuRc4p32FxcRLp81ma36HWKFqhmBHJpF','3AQK2eBgtqWnoknJVnvwUUK9VtvyvZHB2J','13FmEQ1kMSBFnk7cvpZgyctgx9Tg2rbb8D'],5.000001e+09
2020-08-06 14:41:54.803385,['1HSqyCH5mF6jbRcySed5ds8hxLEiNDDevm'],8615598070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
